### Tools

In [28]:
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
from langchain_teddynote import logging

logging.langsmith("edu-CH15-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
edu-CH15-Tools


In [30]:
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

### Python REPL 도구 (built-in)

In [32]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드 실행 도구 생성
python_tool = PythonREPLTool()

In [33]:
# 파이썬 코드 실행
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



In [34]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=False):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [35]:
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

[1, 2, 9, 12, 21, 39]



### Tavily(검색) API Tool

In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    include_domains=["github.io", "wikidocs.net"]
)

In [ ]:
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요."})

[{'title': 'LangGraph Quickstart',
  'url': 'https://langchain-ai.github.io/langgraph/tutorials/introduction/',
  'content': "[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-9-1)Assistant: LangGraph is a library designed to help build stateful multi-agent applications using language models. It provides tools for creating workflows and state machines to coordinate multiple AI agents or language model interactions. LangGraph is built on top of LangChain, leveraging its components while adding graph-based coordination capabilities. It's particularly useful for developing more complex, [...] [](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-21-9)   LangGraph is developed by LangChain, a company known for its tools and frameworks in the AI and LLM space.\n[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-21-10)\n[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-21-11

### DallE (이미지) 생성 도구

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# LLM -> Dalle prompt -> prompt -> image output

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

chain = prompt | llm | StrOutputParser() 

image_prompt = chain.invoke(
    {"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
)

print(image_prompt)

Create a neo-classical painting depicting a group of people in traditional attire, gathered in a grand, classical setting reminiscent of ancient Greece. They are all gazing intently at modern smartphones, their expressions a mix of wonder and distraction. The contrast between the classical architecture, with marble columns and elaborate sculptures, and the sleek, contemporary devices highlights the clash of eras. Include ornate details in the clothing, such as flowing togas and intricate patterns, while the smartphones are sleek and shiny. The lighting should be soft and ethereal, casting gentle shadows. In the background, blend elements of nature, like lush greenery and a serene sky, to emphasize the juxtaposition of technology and classical beauty. The overall tone should be satirical yet respectful, capturing the irony of modern society’s obsession with devices in a historical context.


In [39]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image
import os 

dalle = DallEAPIWrapper(
    model="dall-e-3",
    size="1024x1024",
    quality="standard",
    n=1,
)

query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# 이미지 생성 및 URL 얻기 
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지 표시
Image(url=image_url, width=500)

### 사용자 정의 도구 (Custom Tool)

@tool 데코레이터 

In [43]:
from langchain.tools import tool 

@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [44]:
add_numbers.invoke({"a": 3, "b": 4})

7

### 구글 뉴스기사 검색 도구 

In [45]:
from langchain_teddynote.tools import GoogleNews

news_tool = GoogleNews()

In [46]:
# 최신 뉴스 검색
news_tool.search_latest(k=3)

[{'url': 'https://news.google.com/rss/articles/CBMiZEFVX3lxTFBBaEtFNWY4cXp4WlFnZmVLNkVlTlpPRXViTlJkcVJrVEdWUjFXLVJJNkNzN3FSamxKQTNGZkN4SlBsZng1c0tPLWpVZmRrdHBXV09JQVU0TkFsTnFta1JFT1VYaUs?oc=5',
  'content': '[속보] 경북 산불 다 껐다…7일 만에 서울 75% 면적 초토화 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMid0FVX3lxTE1RVjdaZkZuaWpTRjlwV0xFc2FSRVZnaWUwdjkyWUVPRnBMZExnWTVVQlp5ZlBCRTZGYjNrLVd2TUpuaXlLTzRWWno0bTdicEpNRUs1elFjNGRmNU4wWWM0V3JxbjdfNnVnSVBnY0pCb2hETjhLbURV0gF3QVVfeXFMT3hTVjg0UmNsaXRadjZ0dDhxdVlwbk1VOVlLaWdXekFJSlg0aDQyNE51Y0FzVVh5TE5wTGRRbHMyRGJ6RXZZbG5hcHNON0ZaSHBjYlV3UXdVaVhIWHZQbFZXcFRsaXpzbnR2dHU3QUxYb19PdmFfdFk?oc=5',
  'content': '변협 "헌재 윤 대통령 탄핵심판 조속히 선고해야" - MBC 뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE1sUUJKQkJBM1htbzFOMmgydDZXMDY1cm5sMzBzR05MV3VtLWV2WVRUNUZhU2hpOXg5TGE2V0taTW16TjFJclI3TVRycGphRlZXRjRheTNQRVdZd9IBXEFVX3lxTE1ETlRxR1d5aHdLX2RKZ3J4QjkzRUJYdG96UXZSaE81bnNWWFNfaVdhZE5uZ1dBTVZRODRKaEJOUWlUMG5JaVpWR29WOWRvcFNhOWRnbWxSUnRhd0Fx?oc=5',
  'c

In [47]:
# 키워드 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=2)

[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE5MYk9HMXNxTThFeDRzWllsOEp6MEcxak44RWI0SWVUZFZ4LUppa1BvNHlyalVpT0ZXb2tXdlFRX2NjS2xORmxyWDY3Sm5IeG5ZUXBFNERjQUZZZ9IBVEFVX3lxTE1uRnNtZDl6SjBkZVpSRTVRcjB3WmZPTE5XRktrbUx4bTcwa0NaVUZfOFY4SS1sMnpUbGN4bXUzOGhHc3RrRm4wMHFYZlB4c25ZeXdvQQ?oc=5',
  'content': "삼성전자도 덮친 AI 투자 거품론…6만전자·20만닉스 '위태' - 한국경제"},
 {'url': 'https://news.google.com/rss/articles/CBMiiAFBVV95cUxQUlVHX29lVTY3YnFMRE9YN1pXeGt6bnVKRGpaMDVnMmtWU2FiWEJKZE8yVUl2d0FnNDlFVG1YUUw3R0VDczZ0a0liMi1Ba0Y1Tlh2VW9ENzJ1bTV5cGJTNHRCRkNlT3hfMHV1c0xaMkhNbmEtWVp0anNUYmVsWi12OWItclJidWRX?oc=5',
  'content': '‘AI 투자 비서’가 실시간으로 맞춤형 정보 제공 - 조선비즈'}]